In [15]:
import numpy as np
import random

## Class DNA
'''
This class is elementary object that represents the sequence of 8 numbers forming the position of 8 Queens.
We will generalize this to make it N Queen problem
Attributes:
opm - matrix of N ( dim) distinct numbers that represents Queen positions
dim - To make this a N-Queen problem instead of 8 Queen problem. 
fitness - fitness score of each sequence . FOr the 8 Queen problem, the sequence should be such that no queen should
be on same row or same column or same diagonal with other queen. The way this solution works, index of array is the column,
hence it is always distinct, while generating the sequence of 8 numbers that represent row, we make sure they are distinct
'N' numbers hence First 2 conditions are always satisfied. Fitness score is based on checking how many diagonals have just 
1 queen. For this a set of diagonal_lines is generated that has dim*dim matrix combinations of all possible diagonals.
When we perform element wise multiplication with opm, and sum it up, it should return 1 if only 1 queen position in the entire 
diagonal else the count >1. So fitness score is based on how many diagonals have only 1 queen ( representing correct position)
out of total diagonal combinations.
valid_seq - Considering 'dimXdim' board will have N=dim number of queens, all diagonals will show up each queen position twice.
and so a sequence is valid if fitness score=1 ( 100%) 

Functions : 
create_output_unit_matrix - converts 'N' element array of queen positions to NXN array with 1 in queen positions.
fitness_validity - Calculates fitness of sequence and if it is valid position
crossover - performs cross over of 2 DNA objects
mutate  - performs mutation
'''

class DNA:
    
    opm=[]
    dim=0
    fitness=0.0
    valid_seq=False
    

    #output matrix of N digits
    # generates an array of 'dim' distinct numbers between 0->dim-1
    def __init__(self, dim,generate=True):
        self.opm=[]
        self.dim=dim
        if generate:
            i=0
            while True:
                r=random.randint(0,dim-1)
                if r not in self.opm:
                    self.opm.append(r)
                    i=i+1
                if i==dim:
                    break
            self.opm=np.array(self.opm)
        else:
            self.opm=np.zeros(dim)
            self.dim=dim


    def create_output_unit_matrix(self):
        opm_rows=np.array([0,1,2,3,4,5,6,7])
        opm_cols=self.opm
        indices=np.array((opm_cols,opm_rows)).T
        op_matrix=np.zeros(self.dim*self.dim).reshape(self.dim,self.dim)
        op_matrix[[*indices.T]]=1
        #print(op_matrix)
        return op_matrix

    ## Fitness function that returns a floating point of correct diagonal positions
    ## and if the sequence is a valid N point Queen sequence
    def fitness_validity(self,diagonal_lines):
        valid=False
        vs=0
        opm_unit=self.create_output_unit_matrix()

        for m in diagonal_lines:
            temp=np.sum(np.multiply(m,opm_unit))

            if temp==1:
                vs=vs+1
        #1 position can repeat 2 times in total diagonal lines, hence valid instances will have a dim*2 times of 16 ones
        if vs==(dim*2):
            valid=True
        # fitness_sc = number of valid positions i.e. where only 1 queen in each diagonal/total number of possibilities i.e. dim*2
        fitness_sc=(vs/(dim*2))
        self.fitness=fitness_sc
        self.valid_seq=valid 

    ##CrossOver function
    ## performs cross over of 2 DNA objects to produce a child by randomly selecting the midpoint
    ## based on midpoint, elements 0-midpoint are picked from obj1 and midpoint-dim from obj2
    ## if the combination results in having duplicate elements, it is rejected
    def crossover(self,partner):
        child=DNA(self.dim,False)
        midpoint=random.randint(1,self.dim-1)
        #print("midpoint :",midpoint)
        gene1=self.opm[0:midpoint]
        gene2=partner.opm[midpoint:partner.dim]
        #gene2=partner.opm[np.isin(partner.opm,gene1,invert=True)]
        #validseq=not(np.any(np.isin(gene1,gene2)))
        ch=np.append(gene1,gene2)
        validseq=True if len(np.unique(ch))==8 else False
        if validseq:
            child.opm=ch
        else:
            child=None

        return validseq,child
    
    ##Mutation
    ## for each element, a random number between 0-1 is selected and if it < mutation rate defined
    ## then that element is swapped with another element randomly selected from the sequence
    def mutate(self,mutation_rate):
        for i in self.opm:
            if random.uniform(0, 1) < mutation_rate:
                swap_pos=random.randint(0,self.dim-1)
                self.opm[i],self.opm[swap_pos]=self.opm[swap_pos],self.opm[i]

'''
Function that creates a list of possible diagonal matrices for a N*N board ( dim=N )
Each diagonal matrix has a 1 in diagonal position
'''               
def generateDiagonalLines(dim):
    # create diagonal lines 
    # these diagonal lines will be used to valdiate that given output is result of N queen puzzle
        x=dim
        y=dim
        a = np.arange(x*y).reshape(x,y)


        ## get diagonal values
        diags = [a[::-1,:].diagonal(i) for i in range(-a.shape[0]+1,a.shape[1])]

        # Now back to the original array to get the upper-left-to-lower-right diagonals,
        # starting from the right, so the range needed for shape (x,y) was y-1 to -x+1 descending.
        diags.extend(a.diagonal(i) for i in range(a.shape[1]-1,-a.shape[0],-1))

        # Another list comp to convert back to Python lists from numpy arrays,
        # so it prints what you requested.
    #    print([n.tolist() for n in diags])


        ## get unit matrices on each diagonal line
        diagonal_lines=[]
        for n in diags:
            d=np.asarray([[1 if y in n else 0 for y in x]for x in a])
            diagonal_lines.append(d)

        return diagonal_lines
    
## Class Population
'''
Class to describe a population of DNA objects
Attributes :
    mutation_rate= mutation rate defined for the experiment. Will be set up as part of initial setup
    population= list of DNA objects that form population for the experiment
    mating_pool=list of DNA objects forming the mating pool
    generations=number of iterations of experiment
    finished=if a valid sequence is obtained as part of the population, experiment is set to finished
    perfect_score=fitness score is defined from 0-1 , so perfect score is set to 1
    dim=represents NXN board where dim=N
    diagonal_lines=list of all possible diagonal matrices for N*N experiment
    pop_size=size of population . Will be set up as part of initial set up
    best_index=If a valid sequence is obtained, set up best_index to use it for other purposes
    avg_fitness=average fitness of the entire population 
Functions :
    calcFitness - calculate fitness of all samples in population
    naturalSelection - generate mating pool
    generate - generate next generation of population
    getBest - return sample from the population with best( highest) fitness score
    finished - returns finished status of the experiment
    getGenerations - returns generation/experiment#
    getAverageFitness - calculate average fitness of the population
    showSamples - list top samples from the population
'''

class Population:
    
    mutation_rate=0.0
    population=[]
    mating_pool=[]
    generations=0
    finished=False
    perfect_score=1
    dim=2
    diagonal_lines=None
    pop_size=10
    best_index=0
    avg_fitness=0.0
    
   


    ## constructor method
    def __init__(self,dim,mutationrate,pop_size):
        self.mutation_rate=mutationrate
        self.dim=dim
        self.population=[DNA(dim) for i in range(0,pop_size)]
        self.diagonal_lines=generateDiagonalLines(self.dim)
        self.pop_size=pop_size
        

    # calculate Fitness of all samples of population
    def calcFitness(self):
        for sample in self.population:
            sample.fitness_validity(self.diagonal_lines)
     
    # create mating pool based on fitness score of each sample in the population
    # each sample is duplciated to fitness_Score*100 times
    def naturalSelection(self):
        for sample in self.population:

            for i in range(0,(int(sample.fitness*100))+1):
                self.mating_pool.append(sample)
        random.shuffle(self.mating_pool)

    
    ## create a new generation
    ## select 2 random partners from the mating pool for cross over
    ## if sequence is valid ( returned from DNA.crossover  function) then perform mutation and add to new population
    def generate(self):
        matingpoolsize=len(self.mating_pool)
        new_pop=[]
        new_pop_size=0
        while True:
            partner_a=self.mating_pool[random.randint(0,matingpoolsize-1)]
            partner_b=self.mating_pool[random.randint(0,matingpoolsize-1)]
            #print("parents : ",partner_a.opm, " , ", partner_b.opm)                      
            validseq,child=partner_a.crossover(partner_b)
            if not validseq:
                continue
            else:
                new_pop_size=new_pop_size+1
                child.mutate(self.mutation_rate)
                #print("child :",child.opm)
                new_pop.append(child)
            if new_pop_size==self.pop_size:
                break
        self.generations=self.generations+1
        self.population=new_pop
    
    ## retuens the Best sample with highest fitness_score from the population
    ## if a fitness score of 1 is achieved, sets the experiment's finished status to True
    def getBest(self):
        record=0.0
        index=0
        for sample in self.population:
            
            if sample.fitness > record:
                record=sample.fitness
                self.best_index=index
            if sample.fitness==self.perfect_score:
                self.finished=True
                self.best_index=index
                break
            index=index+1

    ## returns experiment's finished status    
    def finished(self):
        return self.finished
    
    ## returns generation or experiment#
    def getGenerations(self):
        return self.generations
    
    ## calculates average fitness of the entire population
    def getAverageFitness(self):
        total=0
        for sample in self.population:
            total=total+sample.fitness
        #print("total fitness score :", total)
        self.avg_fitness= total/self.pop_size
    
    ## list top samples from the population
    def showSamples(self,sample_size):
        display_limit=min(self.pop_size,sample_size)
        sample=[]
        for i in range(0,display_limit):
            sample.append(self.population[i].opm)
        return sample
 




In [14]:
def main():
    pop=Population(dim,mutationRate,totalPopulation)

    while True:
        pop.calcFitness()
        pop.getAverageFitness()
        pop.getBest()
        print("Generation : ",pop.getGenerations()," Average Fitness :", pop.avg_fitness, "Best sample :", pop.population[pop.best_index].opm, "Fitness score of best sample :", pop.population[pop.best_index].fitness)
        if pop.finished==True:
            print("best queen position")
            print(pop.population[pop.best_index].opm)
            print(pop.population[pop.best_index].fitness)
            break
        pop.naturalSelection()
        pop.generate()

if __name__ == '__main__':
    ## initialization
    dim=8
    mutationRate = 0.1
    totalPopulation = 10
    crossOver = 0.5
    main()


/Users/tanviarora/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Generation :  0  Average Fitness : 0.525 Best sample : [1 4 7 0 2 5 3 6] Fitness score of best sample : 0.75
Generation :  1  Average Fitness : 0.61875 Best sample : [7 4 2 0 5 1 6 3] Fitness score of best sample : 0.75
Generation :  2  Average Fitness : 0.64375 Best sample : [0 4 6 1 3 2 7 5] Fitness score of best sample : 0.75
Generation :  3  Average Fitness : 0.64375 Best sample : [0 3 6 2 5 1 7 4] Fitness score of best sample : 0.875
Generation :  4  Average Fitness : 0.525 Best sample : [7 4 2 0 5 1 6 3] Fitness score of best sample : 0.75
Generation :  5  Average Fitness : 0.6 Best sample : [0 4 1 7 5 2 6 3] Fitness score of best sample : 0.75
Generation :  6  Average Fitness : 0.5375 Best sample : [1 4 0 5 7 2 6 3] Fitness score of best sample : 0.875
Generation :  7  Average Fitness : 0.575 Best sample : [2 7 3 0 5 1 4 6] Fitness score of best sample : 0.875
Generation :  8  Average Fitness : 0.5625 Best sample : [0 4 1 5 7 2 6 3] Fitness score of best sample : 0.75
Generation